In [ ]:
import stela_toolkit as stela
import numpy as np
import matplotlib.pyplot as plt
import pylag
from scipy.signal import convolve

%load_ext autoreload
%autoreload 2

In [ ]:
# Generate original light curve
dt = 10.0
tmax = 10000.0
lc = pylag.SimLightCurve(dt=dt, tmax=tmax, gtzero=True)
lc.error = lc.error+1e-10

# Set parameters
time_lag = 100.0  # Must be integer multiple of dt
shift_bins = int(time_lag // dt)  # Convert to discrete bins

# Create delta function transfer kernel
kernel = np.zeros(shift_bins + 1)
kernel[-1] = 1.0  # Impulse at lag position

# Apply convolution with boundary handling
padded_rate = np.pad(lc.rate, (shift_bins, 0), mode='edge')  # Front-pad with edge values
lagged_rate = convolve(padded_rate, kernel, mode='valid')

# Add independent noise realizations
noise_level = np.mean(lc.error)
resampled_rate1 = lc.rate
resampled_rate2 = lagged_rate

# Create light curve objects with identical time arrays
lc1_py = pylag.LightCurve(t=lc.time, r=resampled_rate1)
lc2_py = pylag.LightCurve(t=lc.time, r=resampled_rate2)

# Verification plot
plt.figure(figsize=(10, 6))
plt.errorbar(lc1_py.time, lc1_py.rate, yerr=lc1_py.error, 
            fmt='o', label="Reference LC", alpha=0.5, color='blue')
plt.errorbar(lc2_py.time, lc2_py.rate, yerr=lc2_py.error, 
            fmt='o', label=f"Lagged by {time_lag}", alpha=0.5, color='red')

# Add vertical markers to verify delay
test_time = 300
plt.axvline(test_time, color='black', linestyle='--', alpha=0.5)
plt.axvline(test_time + time_lag, color='purple', linestyle='--', alpha=0.5)

plt.xlabel("Time")
plt.ylabel("Flux")
plt.legend()
plt.title(f"Convolution-Based Time Lag (τ = {time_lag})")
plt.show()

In [ ]:
### checking unbinned results from consistency with pylag

psd_py = pylag.Periodogram(lc=lc1_py, norm=True)
cs_py = pylag.CrossSpectrum(lc1_py, lc2_py)

# For stela - using SAME time arrays for both light curves
lc1_stela = stela.LightCurve(lc.time, resampled_rate1)
lc2_stela = stela.LightCurve(lc.time, resampled_rate2)
psd_stela = stela.PowerSpectrum(lc1_stela, norm=True)
cs_stela = stela.CrossSpectrum(lc1_stela, lc2_stela)
lfs_stela = stela.LagFrequencySpectrum(lc1_stela, lc2_stela)

plt.figure(figsize=(8, 5))
plt.errorbar(psd_stela.freqs, psd_stela.powers, fmt='o', label='stela')
plt.errorbar(psd_py.freq, psd_py.periodogram, psd_py.error, fmt='o', label='pylag')
plt.yscale('log')
plt.xscale('log')
plt.ylabel('Power')
plt.xlabel('Frequency')
plt.legend()

plt.figure(figsize=(8, 5))
plt.errorbar(cs_stela.freqs, cs_stela.cs, fmt='o', label='stela')
plt.errorbar(cs_py.freq, cs_py.crossft, fmt='o', label='pylag')
plt.xscale('log')
plt.xlabel('Frequency')
plt.ylim([-50, 200])
plt.legend()

plt.figure(figsize=(8, 5))
plt.errorbar(lfs_stela.freqs, lfs_stela.lags, lfs_stela.lag_errors, fmt='o', label='stela')
plt.errorbar(cs_py.freq, np.angle(cs_py.crossft)/(2*np.pi*cs_py.freq), fmt='o', label='pylag')
plt.xscale('log')
plt.xlabel('Frequency')
plt.ylim([-200, 200])
plt.legend()

In [ ]:
### checking binned results from consistency with pylag
bins_py = pylag.LogBinning(0.0001, 0.05, num=10)
psd_py = pylag.Periodogram(lc=lc1_py, norm=True).bin(bins_py)
cs_py = pylag.CrossSpectrum(lc1_py, lc2_py).bin(bins_py)

num_bins = 10
psd_stela = stela.PowerSpectrum(lc1_stela, num_bins=10, bin_type='log', norm=True)
psd_stela_unbinned = stela.PowerSpectrum(lc1_stela, norm=True)
cs_stela = stela.CrossSpectrum(lc1_stela, lc2_stela, num_bins=10,)
lfs_stela = stela.LagFrequencySpectrum(lc1_stela, lc2_stela, num_bins=10)
print(lfs_stela.cohs)
print(lfs_stela.lag_errors)
cs_stela_unbinned = stela.CrossSpectrum(lc1_stela, lc2_stela)
lfs_stela_unbinned = stela.LagFrequencySpectrum(lc1_stela, lc2_stela)

plt.figure(figsize=(8, 5))
plt.errorbar(psd_stela.freqs, psd_stela.powers, xerr=psd_stela.freq_widths, yerr=psd_stela.power_errors, fmt='o', label='stela')
plt.errorbar(psd_stela_unbinned.freqs, psd_stela_unbinned.powers, fmt='o', label='stela unbinned')
plt.errorbar(psd_py.freq, psd_py.periodogram, xerr=psd_py.freq_error, yerr=psd_py.error, fmt='o', label='pylag')
plt.yscale('log')
plt.xscale('log')
plt.ylabel('Power')
plt.xlabel('Frequency')
plt.legend()

plt.figure(figsize=(8, 5))
plt.errorbar(cs_stela.freqs, cs_stela.cs, xerr=psd_stela.freq_widths, yerr=cs_stela.cs_errors, fmt='o', label='stela')
plt.errorbar(cs_stela_unbinned.freqs, cs_stela_unbinned.cs, yerr=cs_stela_unbinned.cs_errors, fmt='o', label='stela unbinned')
plt.xscale('log')
plt.xlabel('Frequency')
plt.ylim([-50, 200])
plt.legend()

plt.figure(figsize=(8, 5))
plt.errorbar(lfs_stela.freqs, lfs_stela.lags, xerr=psd_stela.freq_widths, yerr=lfs_stela.lag_errors, fmt='o', label='stela')
plt.errorbar(lfs_stela_unbinned.freqs, lfs_stela_unbinned.lags, lfs_stela_unbinned.lag_errors, fmt='o', label='stela unbinned')

plt.xscale('log')
plt.xlabel('Frequency')
plt.ylim([-200, 200])
plt.legend()